In [570]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, FunctionTransformer, OrdinalEncoder
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier

## Modelagem

Para construção da máquina preditiva, é de total importância que o modelo seja alimentado com dados de qualidade. Dada a análise exploratória, devemos contruir um modelo capaz de predizer com maior assertividade do que um simples método de sempre retornar o valor de `No Failure`, pois devido ao desbalanceamento, esses simples chute terá uma alta pontuanção, a depender da métrica utilizada.


Para a base de dados em questão, temos um problema de *multiclass classification*. E, ainda levando em consideração a eda, inicialmente, devido a simplicidade da relação dos dados, e as poucas quantidades de features e amostras, podemos primeiramente avaliar a performance de modelos lineares, como uma regressão logística ou SVM, e posteriormente, verificar como modelos baseados em árvore se comportam, como Random Forest ou Ensembles. Dado o tipo de problema de multiclass, pode ser válido a análise de modelos específicos para tal, como OneVsRestClassifier

### Pre-processamento dos Dados

- Carregamento da base de treinio

In [540]:
df = pd.read_csv("desafio_manutencao_preditiva_treino.csv")
df.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
0,1,M14860,M,298.1,308.6,1551,42.8,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,No Failure
2,5,L47184,L,298.2,308.7,1408,40.0,9,No Failure
3,6,M14865,M,298.1,308.6,1425,41.9,11,No Failure
4,7,L47186,L,298.1,308.6,1558,42.4,14,No Failure


- Carregamento da base de teste

In [541]:
x_test = pd.read_csv("desafio_manutencao_preditiva_teste.csv")
x_test.head()

,udi,product_id,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min
0,446,L47625,L,297.5,308.6,1793,26.7,70
1,7076,L54255,L,300.7,310.5,1536,47.4,192
2,1191,L48370,L,297.2,308.4,1460,42.1,41
3,2618,L49797,L,299.4,309.1,1670,35.9,68
4,5067,L52246,L,304.1,313.1,1550,30.9,9


#### Seleção das variáveis

1 - Vamos eliminar as variáveis que não fornecem informações relevantes para o problema, como as variáveis de identificação.

In [542]:
df = df.drop(columns=['udi', 'product_id'])
df.head()

,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type
0,M,298.1,308.6,1551,42.8,0,No Failure
1,L,298.2,308.7,1408,46.3,3,No Failure
2,L,298.2,308.7,1408,40.0,9,No Failure
3,M,298.1,308.6,1425,41.9,11,No Failure
4,L,298.1,308.6,1558,42.4,14,No Failure


In [543]:
x = df.drop('failure_type', axis=1)
y = df[['failure_type']]

x.shape, y.shape

((6667, 6), (6667, 1))

Codificação da nossa variável alvo

In [544]:
y_encoder = LabelEncoder()
y_train = pd.DataFrame(y_encoder.fit_transform(y))
y_train

/home/lineker/Ufop/Estagio/Indicium/indicium/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0
0,1
1,1
2,1
3,1
4,1
...,...
6662,1
6663,1
6664,1
6665,1


#### Outliers

Com as informações obtidas das análises gráficas, vimos que as as variáveis `rotational_speed_rp` e `torque_nm` possuem alguns dados que extrapolam os valores máximos dos boxplots, contudo, como ambas as variáveis são normalmente distribuídas, podemos usar o método de z-score para tratar possíveis outliers. Onde iremos medir quão longe nossos dados estão da média, e para cada dado que ultrapassar um determinado threshold, iremos removê-lo.

In [545]:
def remove_outliers(df, column_name, threshold=3):
    mean = df[column_name].mean()
    std = df[column_name].std()
    z_scores = (df[column_name] - mean) / std
    df = df[(np.abs(z_scores) < threshold)]
    return df

#### Pipeline para encodings e scaling

A etapa de codificação será aplicada somente a variávels categórica `type`, pois as variáveis de identificação dos produtos serão dropadas posteriormente, não sendo interessantes para o problema atual, além de que, nossa variável `type`, possui um carater ordinal implícito. Para a etapa de escalonamento das variáveis, usaremos o método min-max normalization, já que nossa base total não possui uma distribuição normal, e também serão retirados os outliers.

In [546]:
categorical_transforms = Pipeline([
    ('ordinal_encoder', OrdinalEncoder())
    ]
)
numerical_transforms = Pipeline([
    ('scaler', MinMaxScaler())
])

In [547]:
categorical_columns = x.select_dtypes(include=['object']).columns.values.tolist()
numerical_columns = x.select_dtypes(exclude=['object']).columns.values.tolist()
print(f"Atributos categóricos: {categorical_columns}")
print(f"Atributos numéricos: {numerical_columns}")

Atributos categóricos: ['type']
Atributos numéricos: ['air_temperature_k', 'process_temperature_k', 'rotational_speed_rpm', 'torque_nm', 'tool_wear_min']


2 - Aqui, usaremos algumas informações do dicionário de dados para produzir novos atributos para nossa base de dados.

In [548]:
df.assign(
    too_hot=lambda f: f['process_temperature_k'] - f['air_temperature_k'],
    too_power=lambda f: ((f['rotational_speed_rpm'] * 2 * np.pi) / 60) * f['torque_nm'],
    too_used=lambda f: f['tool_wear_min'] * f['torque_nm']
    )

,type,air_temperature_k,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,failure_type,too_hot,too_power,too_used
0,M,298.1,308.6,1551,42.8,0,No Failure,10.5,6951.590560,0.0
1,L,298.2,308.7,1408,46.3,3,No Failure,10.5,6826.722724,138.9
2,L,298.2,308.7,1408,40.0,9,No Failure,10.5,5897.816608,360.0
3,M,298.1,308.6,1425,41.9,11,No Failure,10.5,6252.554779,460.9
4,L,298.1,308.6,1558,42.4,14,No Failure,10.5,6917.703247,593.6
...,...,...,...,...,...,...,...,...,...,...
6662,L,298.8,308.3,1634,27.9,12,No Failure,9.5,4774.027028,334.8
6663,M,298.8,308.4,1604,29.5,14,No Failure,9.6,4955.129373,413.0
6664,H,298.9,308.4,1632,31.8,17,No Failure,9.5,5434.703963,540.6
6665,H,299.0,308.7,1408,48.5,25,No Failure,9.7,7151.102638,1212.5


In [549]:
# def get_difference_temperature(df):
#     df['too_hot'] = df['process_temperature_k'] - df['air_temperature_k']
#     return df

# def get_power(df):
#     df['power'] = ((df['rotational_speed_rpm'] * 2 * np.pi) / 60) * df['torque_nm']
#     return df
    
# def get_usage(df):
#     df['usage'] = df['tool_wear_min'] * df['torque_nm']
#     return df

In [550]:
# feature_engineering = ColumnTransformer([
#     ('difference_temperature', FunctionTransformer(get_difference_temperature, validate=False),
#      ['process_temperature_k', 'air_temperature_k']),
#     ('power', FunctionTransformer(get_power, validate=False), 
#      ['rotational_speed_rpm', 'torque_nm']),
#     ('usage', FunctionTransformer(get_usage, validate=False),
#      ['tool_wear_min', 'torque_nm'])
# ])

#### Balanceamento das classes

Para realizar uma correta implementação do balanceamento das classes, é necessário a implementação manual do cross validatopn, pois é necessário que, nos folds de validação, a distribuição das classes seja mantida, o balanceamento só pode ser aplicado na fase de treinamento para melhor generalização do modelo.

In [551]:
# def cross_validation(modelo, x, y, oversampling=False):
    
#     kfold = KFold(n_splits=10)
    
#     metrics = {"precision": [], "recall": [], "f1_score": []}
    
#     for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(x)):
#         x_split_treino = x.iloc[idx_treino, :]
#         y_split_treino = y.iloc[idx_treino, :]
        
#         # oversamplig
#         if oversampling:
#             sm = SMOTE(random_state=24)            
#             x_split_treino, y_split_treino = sm.fit_resample(x_split_treino, y_split_treino)
            
#         modelo.fit(x_split_treino, y_split_treino.values.flatten())
        
#         x_split_validacao = x.iloc[idx_validacao, :]
#         y_split_validacao = y.iloc[idx_validacao, :].values
        
#         # validação realizada sem o oversampling
#         predicoes = modelo.predict(x_split_validacao)
        
#         precision, recall, f_score, _ = precision_recall_fscore_support(
#             y_split_validacao, 
#             predicoes,
#             average='macro',
#             zero_division=1)
#         metrics['precision'].append(precision)
#         metrics['recall'].append(recall)
#         metrics['f1_score'].append(f_score)
        
    
#     return metrics

#### Testando Modelos

In [552]:
pre_processamento = ColumnTransformer(transformers=[
    ('categoricas', categorical_transforms, categorical_columns),
    ('numericas', numerical_transforms, numerical_columns)
])
pre_processamento

ColumnTransformer(transformers=[('categoricas',
                                 Pipeline(steps=[('ordinal_encoder',
                                                  OrdinalEncoder())]),
                                 ['type']),
                                ('numericas',
                                 Pipeline(steps=[('scaler', MinMaxScaler())]),
                                 ['air_temperature_k', 'process_temperature_k',
                                  'rotational_speed_rpm', 'torque_nm',
                                  'tool_wear_min'])])

In [553]:
pipeline_reg_log = Pipeline([
    ('pre_processamento', pre_processamento),
    ('model', BalancedBaggingClassifier(estimator=DecisionTreeClassifier(),
                                        sampling_strategy='all',
                                        replacement=False,
                                        random_state=25))
])

In [565]:
pipeline_reg_log.fit(x_train, y_train.values.flatten())

Pipeline(steps=[('pre_processamento',
                 ColumnTransformer(transformers=[('categoricas',
                                                  Pipeline(steps=[('ordinal_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['type']),
                                                 ('numericas',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['air_temperature_k',
                                                   'process_temperature_k',
                                                   'rotational_speed_rpm',
                                                   'torque_nm',
                                                   'tool_wear_min'])])),
                ('model',
                 BalancedBaggingClassifier(estimator=DecisionTreeClassifier(),
                                           random_state=25,
                                           sampling_strategy='all'))])

In [566]:
cross_val = KFold(n_splits=10, shuffle=True, random_state=25)
cross_val

KFold(n_splits=10, random_state=25, shuffle=True)

In [580]:
scores = cross_val_score(pipeline_reg_log, x, y.values.flatten(), cv=10, scoring='recall_macro')
scores

array([0.76466529, 0.67279158, 0.63254831, 0.67362836, 0.55113009,
       0.66081519, 0.65288454, 0.6630255 , 0.60832778, 0.67183836])

In [571]:
predictions = pipeline_reg_log.predict(x_test)